# Construct a figure showing sensitivity analysis results:

example notebook to create a figure to show sensitivity analysis results by Beo

In [338]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

In [339]:
# name of the file containing the data in .csv format
data_file = 'data/model_results_sensitivity_analysis_beowawe.csv'

Define the columns that contain the input parameters and model results:

In [340]:
# name of the index column:
index_column = 'index'

# index row for the base case model
base_case_model = 0

# list the parameters that you want to include
sensitivity_params = ['fault_widths', 'fault_fluxes', 'exhumation_rate', 
                      'thermal_gradient', 'ra', 'durations_yr']

# list the first and last index row for each parameter
#param_rows = [[1, 3], [4, 6]]

# show the percentage change or the ratio of each parameter comapred to the base case:
x_axis_scale = 'fraction'

# what to plot on the y axis
result_params = ['width_partial_reset', 'width_full_reset']

# base case runtime: 
default_runtime = 2e5

Some parameters that control how the figure looks:

In [341]:
# markers for the different parameters
markers = ['o', 's', 'v', '^', 'd', 'h']
base_case_marker = '*'
marker_size = 50
base_case_marker_size = 150

# labels for the legend
labels = ['Width fault zone (m)', r'Flux (m$^2$a$^{-1}$)', r'Exhumation rate ($m yr^{-1}$)', 
          r'Thermal gradient ($\^{\circ}C m^{-1}$)', 'Aerodynamic resistance (unit?)', 'Duration (yr)']

# label on x and y-axis
xlabel = 'Parameter value / base value'
ylabels = ['Width partial reset zone (m)', 'Width full reset zone (m)']

# limits of the x and y-axis
xlims = [[0, 2.5], [0, 11.0]]
ylims = [[0, 150], [0, 3600]]

# colors for the different parameters
# use standard colors like this:
#colors = ['blue', 'orange']
# or check here for nicer RGB color schemes: http://colorbrewer2.org/#type=qualitative&scheme=Set2&n=3
colors = [[102/255., 194/255., 165/255.], [252/255., 141/255., 98/255.], [141/255., 160/255., 203/255.], 
          [231/255., 138/255., 195/255.], [166/255., 216/255., 84/255.], [255/255., 217/255., 47/255.]]

legend_location = 'lower right'
legend_font_size = 'small'
legend_ncol = 3

# figure size (in inches):
figsize = (8, 7)

Read the data file:

In [ ]:
# read the datafile using pandas:
# cols to read in dataframe:
#cols = ['runtime_yr', 'fault_widths', 'fault_fluxes', 'exhumation_rate', 'thermal_gradient', 'ra', 'durations', 
#        'x_min_partial_reset_surface', 'x_max_partial_reset_surface', 'x_min_full_reset_surface', 'x_max_full_reset_surface']

#cols = ['runtime_yr', 'fault_widths', 'fault_fluxes', 'exhumation_rate', 'thermal_gradient', 'ra', 'durations',#, 
#       'x_min_partial_reset_surface']#, 'x_max_partial_reset_surface', 'x_min_full_reset_surface', 'x_max_full_reset_surface']


df = pd.read_csv(data_file, usecols=cols)
#df = df.set_index(index_column)
df

calculate the width of the partial and full reset zones:

In [ ]:
df['width_partial_reset'] = df['x_max_partial_reset_surface'] - df['x_min_partial_reset_surface']
df['width_full_reset'] = df['x_max_full_reset_surface'] - df['x_min_full_reset_surface']

df.loc[df['width_partial_reset'].isnull(), 'width_partial_reset'] = 0
df.loc[df['width_full_reset'].isnull(), 'width_full_reset'] = 0

convert fluid flux column to array

In [ ]:
list_cols = ['fault_fluxes', 'durations', 'fault_widths']

for lcol in list_cols:
    fl = df[lcol].values.tolist()
    fla = np.array([float(str(f).strip('[').strip(']')) for f in fl])
    df[lcol] = fla

df['durations_yr'] = df['durations'] / (365.25*24*3600)

find out in which rows each particular parameter varies

In [ ]:
param_values = []
result_values = []

for param in sensitivity_params:
    if param == 'durations_yr':
        dfr = df.loc[df['durations_yr'] != default_runtime]
        param_values.append(dfr[param])
        
        r = []
        for result_param in result_params:
            ri = dfr.loc[ind, result_param]
            r.append(ri)
        result_values.append(r)
        
    else:
        dfb = df.loc[df['runtime_yr'] == default_runtime]
        base_ind = dfb.index[0]
        ind = dfb[param] != dfb.loc[base_ind, param]
        
        p = dfb.loc[ind, param]
        param_values.append(p)
        r = []
        for result_param in result_params:
            ri = dfb.loc[ind, result_param]
            r.append(ri)
        result_values.append(r)  

find the base case params, this is assumed to be the first model run in the .csv file:

In [ ]:
base_params = []
base_results = []
for param in sensitivity_params:
    dfb = df.loc[df['runtime_yr'] == default_runtime]
    base_ind = dfb.index[0]
    base_params.append(dfb.loc[base_ind, param])
    r = []

for result_param in result_params:
    ri = dfb.loc[base_ind, result_param]
    #r.append(ri)
    base_results.append(ri)
    
print('base case: ', base_params, base_results)

show the parameters and model results:

In [ ]:
for param, param_val, result_var in zip(sensitivity_params, param_values, result_values):
    print('-------')
    print(param)
    print (param_val, result_var)

make the figure

In [ ]:
fig, panels = pl.subplots(2, 2, figsize=figsize)

panelsr = panels.ravel()

for j, xlim, ylim in zip(itertools.count(), xlims, ylims):
    for i, panel in enumerate(panels[j]):
        for param, param_value, base_param, result_value, color, marker, label in zip(sensitivity_params, param_values, base_params, result_values, colors, markers, labels):

            if i > 0 or j > 0:
                label = ''

            if x_axis_scale is 'fraction':
                x_values = param_value.values / base_param
                x_values = np.append(x_values, [1.0])
            
            y_values = result_value[i].values
            
            # add base case values:
            y_values = np.append(y_values, [base_results[i]])
            
            # make sure values are sorted
            s = np.argsort(x_values)
            
            panel.plot(x_values[s], y_values[s], color=color, label='', zorder=100)
            panel.scatter(x_values, y_values, color='None', edgecolor=color, marker=marker, s=marker_size, label=label, zorder=101)

        # show the base case value:
        if i == 0  and j == 0:
            blabel = 'base case'
        else:
            blabel = ''
        panel.scatter([1.0], [base_results[i]],
                      color='gray', edgecolor='black', marker=base_case_marker,
                      s=base_case_marker_size, label=blabel, zorder=102)

    for panel, ylabel in zip(panels[j], ylabels):
        panel.set_ylabel(ylabel)
        panel.set_xlim(xlim)
        panel.set_ylim(ylim)

for panel in panelsr:
    panel.set_xlabel(xlabel)
    
# make the figure look nicer:
for panel in panelsr:
    panel.grid(False)
    panel.spines['top'].set_visible(False)
    panel.spines['right'].set_visible(False)

# add a legend
fig.subplots_adjust(bottom=0.17)
fig.legend(loc='lower center', fontsize=legend_font_size, ncol=legend_ncol, frameon=False)

and save the figure:

In [ ]:
# svae the figure as pdf and png file
fn = data_file[:-4] + '_sensitivity_fig.pdf'
print('done, saving figure as %s' % fn)
fig.savefig(fn)